In [139]:
import pandas as pd
import json
import numpy as np
import re


In [140]:
df = pd.read_csv('dataset/agg/recommendations.csv', )

In [141]:
# Then, merge all "recommendations_str.*" columns back into a single "recommendations_str" column
df['recommendations_str'] = df[df.columns[1:]].apply(lambda x: ' '.join(x.astype(str)), axis=1)

# You can then drop the redundant columns
df = df[['user', 'recommendations_str']]

In [142]:
df['recommendations_str'] = df['recommendations_str'].str.replace(r'[\\\"\[\]]', '', regex=True)

In [143]:
df['recommendations_str'] = df['recommendations_str'].str.replace('} {', "}, {")
df['recommendations_str'] = df['recommendations_str'].str.replace("rating", ',"rating"')
df['recommendations_str'] = df['recommendations_str'].str.replace("item", '"item"')
df['recommendations_str']

0         {"item":43342 ,"rating":20.765831}, {"item":13...
1         {"item":34006 ,"rating":11.692207}, {"item":44...
2         {"item":20706 ,"rating":11.140845}, {"item":44...
3         {"item":43342 ,"rating":41.19859}, {"item":394...
4         {"item":20706 ,"rating":15.044108}, {"item":34...
                                ...                        
206204    {"item":46514 ,"rating":47.255756}, {"item":44...
206205    {"item":43839 ,"rating":8.834701}, {"item":478...
206206    {"item":24609 ,"rating":10.420514}, {"item":43...
206207    {"item":3253 ,"rating":21.97093}, {"item":1769...
206208    {"item":33752 ,"rating":8.364368}, {"item":440...
Name: recommendations_str, Length: 206209, dtype: object

In [144]:
import ast

recommendations = df['recommendations_str'].apply(lambda x: ast.literal_eval(x))

print(recommendations)

0         ({'item': 43342, 'rating': 20.765831}, {'item'...
1         ({'item': 34006, 'rating': 11.692207}, {'item'...
2         ({'item': 20706, 'rating': 11.140845}, {'item'...
3         ({'item': 43342, 'rating': 41.19859}, {'item':...
4         ({'item': 20706, 'rating': 15.044108}, {'item'...
                                ...                        
206204    ({'item': 46514, 'rating': 47.255756}, {'item'...
206205    ({'item': 43839, 'rating': 8.834701}, {'item':...
206206    ({'item': 24609, 'rating': 10.420514}, {'item'...
206207    ({'item': 3253, 'rating': 21.97093}, {'item': ...
206208    ({'item': 33752, 'rating': 8.364368}, {'item':...
Name: recommendations_str, Length: 206209, dtype: object


In [146]:
df['recommendations'] = recommendations

In [150]:
df = df[['user','recommendations']]

In [157]:
df = df.set_index('user').explode('recommendations').reset_index()

In [158]:
df = pd.concat([df.drop(['recommendations'], axis=1), df['recommendations'].apply(pd.Series)], axis=1)

In [159]:
df

,user,item,rating
0,1,43342.0,20.765831
1,1,13824.0,14.240823
2,1,1571.0,14.128424
3,1,28934.0,13.554399
4,1,47819.0,13.105035
...,...,...,...
2062085,206202,32140.0,5.662667
2062086,206202,4557.0,5.652735
2062087,206202,38697.0,5.584359
2062088,206202,47890.0,5.553699


In [160]:
df.to_csv("recommendations_cleaned.csv")